In [18]:
import numpy as np
import random
from scipy.spatial.distance import squareform, pdist
from skimage.util import img_as_float

# a = np.zeros(3)
# b = a
# b[0] = 1
# print(a)

a = np.array([[[1,1,3],
              [1,1,2],
              [1,1,4]],

              [[1,1,3],
              [1,1,2],
              [2,1,4]],
                
              [[1,1,3],
              [1,1,2],
              [3,1,4]]])

b=np.array([[1,1,3],
              [1,1,2],
              [1,1,4]])


# b[:, 0] = np.array([[3],[3],[3]])

# print(np.sum(a, axis = 1))
# print(b)
# print(a[:, :, 2])

# b = np.array([1])

# print(a * [2, 3])

# print(a-b)

H = 3
W = 4
# print(np.mgrid[0:H, 0:W])
# print(np.dstack(np.mgrid[0:H, 0:W]))

mask = np.array([True, False,True])
print(a[mask])


[[[1 1 3]
  [1 1 2]
  [1 1 4]]

 [[1 1 3]
  [1 1 2]
  [3 1 4]]]


In [19]:
A = np.array([[1, 2, 3, 4, 5],
              [1, 2, 3, 4, 5]])

res = np.where(A > 3)

# print(res)
# print((A>3) * A)

print(A.T)
print((A.T)[A[0] < 3])
print(A[0] < 3)

[[1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]]
[[1 1]
 [2 2]]
[ True  True False False False]


In [20]:
features = np.zeros((12, 3))

xy_coords = np.dstack(np.mgrid[0:H, 0:W]).reshape(-1, 2)

print(xy_coords)


dist = np.sqrt(np.sum(xy_coords ** 2, axis=-1))

print(dist)
features[:, 1] = dist

print(features)

[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [1 0]
 [1 1]
 [1 2]
 [1 3]
 [2 0]
 [2 1]
 [2 2]
 [2 3]]
[0.         1.         2.         3.         1.         1.41421356
 2.23606798 3.16227766 2.         2.23606798 2.82842712 3.60555128]
[[0.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         2.         0.        ]
 [0.         3.         0.        ]
 [0.         1.         0.        ]
 [0.         1.41421356 0.        ]
 [0.         2.23606798 0.        ]
 [0.         3.16227766 0.        ]
 [0.         2.         0.        ]
 [0.         2.23606798 0.        ]
 [0.         2.82842712 0.        ]
 [0.         3.60555128 0.        ]]


In [21]:
from skimage import io

img = io.imread('./data/imgs/cat_bed.jpg')
num_segments = 8

H, W, C = img.shape


features = np.zeros((3, C))


color = img_as_float(img)
color = color.reshape(-1, 3)

# print(color.shape)
# print((color * [0.299, 0.587, 0.114]).shape)
# print(np.sum(color * [0.299, 0.587, 0.114], axis = -1).shape)
print(features[:, 0].shape)
features[:, 0] = np.sum(a * [0.299, 0.587, 0.114], axis = -1)
# print(np.sum(color * [0.299, 0.587, 0.114], axis = -1))
print(features)

(3,)


ValueError: could not broadcast input array from shape (3,3) into shape (3,)

In [ ]:
def zero_pad(image, pad_height, pad_width):
    H, W = image.shape
    out = None
    out = np.zeros((H+2*pad_height, W+2*pad_width))
    out[pad_height : pad_height + H, pad_width : pad_width + W] = image
    return out

def conv_fast(image, kernel):
    Hi, Wi = image.shape
    Hk, Wk = kernel.shape
    out = np.zeros((Hi, Wi))
    image = zero_pad(image, pad_height=Hk // 2, pad_width=Wk // 2)
    kernel = np.flipud(np.fliplr(kernel))
    for i in range(Hi):
        for j in range(Wi):
            out[i][j] += np.sum(image[i : i + Hk, j :  j + Wk] * kernel)
    out = out / np.sum(out)
    return out

# laplacian (use green channel since green channel is most similar to brightness)
laplacian_kernel = np.array([[-1,-1,-1],
                                [-1, 8,-1],
                                [-1,-1,-1]])
img_laplacian = conv_fast(img[:,:,1], laplacian_kernel)

# sum up
sum_kernel = np.ones((5,5))
sum_kernel = sum_kernel / 25
img_lap_sum = conv_fast(img_laplacian, sum_kernel)

# set as feature
features[:, 3] = img_laplacian.reshape(-1)

In [ ]:
# Generate random data points for clustering

# Set seed for consistency
np.random.seed(0)

# Cluster 1
mean1 = [-1, 0]
cov1 = [[0.1, 0], [0, 0.1]]
X1 = np.random.multivariate_normal(mean1, cov1, 100)

# Cluster 2
mean2 = [0, 1]
cov2 = [[0.1, 0], [0, 0.1]]
X2 = np.random.multivariate_normal(mean2, cov2, 100)

# Cluster 3
mean3 = [1, 0]
cov3 = [[0.1, 0], [0, 0.1]]
X3 = np.random.multivariate_normal(mean3, cov3, 100)

# Cluster 4
mean4 = [0, -1]
cov4 = [[0.1, 0], [0, 0.1]]
X4 = np.random.multivariate_normal(mean4, cov4, 100)

# Merge two sets of data points
X = np.concatenate((X1, X2, X3, X4))

In [ ]:
from scipy.spatial.distance import squareform, cdist

features = X
N, D = features.shape
k = 5
num_iters = 1
idxs = np.random.choice(N, size=k, replace=False)
centers = features[idxs]
assignments = np.zeros(N, dtype=int)

e = 0.01
w=0.25

N, D = features.shape

# Assign each point to its own cluster
assignments = np.zeros(N, dtype=np.uint32)

###
# Normalization
features = features / np.max(features, axis=0)

mode_array = np.zeros(features.shape)
###

for n in range(N):
    feat = features[n]
    ### YOUR CODE HERE
    # init center
    mode = feat.copy()

    while True:
        # find pixels in window
        # print(np.max(np.abs(features - mode), axis=-1).shape)
        in_window_idx = np.where((np.max(np.abs(features - mode), axis=-1)) < (w/2))
        in_window_pts = features[in_window_idx]

        # find new centroid
        new_centroid = np.mean(in_window_pts, axis=0)

        # check shift
        if np.max(np.abs(new_centroid - mode)) < e:
        # shift = np.linalg.norm(mode - new_centroid)
        # if shift < e:
            mode_array[n] = new_centroid
            break

        # set new mode
        mode = new_centroid
    ### END YOUR CODE
        
### group
cluster_cnt = 1
for i, mode in enumerate(mode_array):
    if assignments[i] == 0:         # not clustered
        cluster_idx = np.where(np.max(np.abs(mode_array - mode), axis=-1) < (w/2))
        assignments[cluster_idx] = cluster_cnt
        cluster_cnt += 1

assignments -= 1
print(np.max(assignments))

33


In [ ]:
from scipy.spatial.distance import squareform, cdist

features = X
N, D = features.shape
k = 5
num_iters = 1
idxs = np.random.choice(N, size=k, replace=False)
centers = features[idxs]
assignments = np.zeros(N, dtype=int)

e = 0.01
w=0.25

for n in range(1):
    feat = features[n]
    ### YOUR CODE HERE
    # normalization
    feat /= np.max(feat)

    # init mode
    mode = feat


    # find mode
    while True:
        # place window
        # print(np.abs(features[:, 0] - mode[0]))
        # print(np.abs(features[:, 0] - mode[0]) <= (w / 2))
        window_idxs = np.where((np.abs(features[:, 0] - mode[0]) <= w / 2) & (np.abs(features[:, 1] - mode[1]) <= w / 2))

        # cal centroid
        centroid = np.mean(features[window_idxs], axis=0)

        # check update
        if np.all(np.abs(mode- centroid) < e):
            break
        else:
            mode =  centroid

    # assign to an existing or new cluster
    # print((np.abs(features[:, 0] - mode[0]) <= w / 2))
    possible_similar_mode_idxs = np.where((np.abs(features[:, 0] - mode[0]) <= w / 2) & (np.abs(features[:, 1] - mode[1]) <= w / 2))
    print(possible_similar_mode_idxs[0][0])


    ### END YOUR CODE

0


In [ ]:
# np.mgrid[0:4, 0:3]
# np.arange(20).reshape(5, 4)
# np.indices((2, 3))
# np.mgrid[0:2, 0:3]

a = np.array((1,2,3))
b = np.array((2,3,4))


print(a)
print(b)
print(np.dstack((a,b)))

img = np.array(range(60)).reshape(5, 4, 3)
H, W, C = img.shape
color = img_as_float(img)
features = np.zeros((H*W, C+2))

print(color.shape)
print(np.mgrid[0:H, 0:W].shape)

### YOUR CODE HERE
features = np.concatenate((color, np.mgrid[0:W, 0:H]), axis=0)
features = features.reshape(-1, C+2)

print(img)

[1 2 3]
[2 3 4]
[[[1 2]
  [2 3]
  [3 4]]]
(5, 4, 3)
(2, 5, 4)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 3 and the array at index 1 has size 5

In [ ]:
from scipy.spatial.distance import squareform, cdist

features = X
N, D = features.shape
k = 5
num_iters = 1
idxs = np.random.choice(N, size=k, replace=False)
centers = features[idxs]
assignments = np.zeros(N, dtype=int)

### YOUR CODE HERE
# choose first center
centers = np.zeros((k, 2))
centers[0] = np.random.choice(N, size=1)



# choose other centers
for i in range(1, k):
    dist = cdist(features, centers[:i])

    dist = np.min(dist, axis=-1)      # D(x) for each data

    dist = dist ** 2
    dist /= np.sum(dist)        # normalize as probability

    # print(dist)
    # print("\n")

    centers[i] = np.random.choice(N, size=1, p=dist)    # will not choose the same one since the probability is 0

print(centers)

# K-means

for _ in range(num_iters):
    assignments_prev = assignments.copy()

    # find dist
    dists = cdist(features, centers)

    # cluster
    assignments = np.argmin(dists, axis=-1)

    for c in range(k):
        centers[c] = np.mean(features[assignments == c], axis=0)

    # check assignment
    if np.array_equal(assignments_prev, assignments):
        break


[[246. 246.]
 [ 22.  22.]
 [292. 292.]
 [314. 314.]
 [298. 298.]]


In [ ]:
from scipy.spatial.distance import squareform, cdist

features = X
N, D = features.shape
k = 5
num_iters = 1
idxs = np.random.choice(N, size=k, replace=False)
centers = features[idxs]
assignments = np.zeros(N, dtype=int)

for n in range(num_iters):

    print(centers)
    print("\n")

    ### YOUR CODE HERE
    # find dist
    dists = cdist(features, centers)
    print(dists)
    print("\n")

    # cluster
    assignments = np.argmin(dists, axis=-1)

    # print(assignments)
    # print("\n")

    # compute new center
    centers = np.zeros((k, D))
    cluster_cnt = np.zeros((k, 1))
    for i in range(N):
        centers[assignments[i]] += features[i]
        cluster_cnt[assignments[i]] += 1
    
    centers = centers / cluster_cnt
    print(centers)
    print("\n")



[[-0.04171317 -0.87200308]
 [ 0.07134372 -0.85778855]
 [-0.40942631 -0.3090424 ]
 [ 1.25757378  0.01863391]
 [-0.95986687  0.1271202 ]]


[[3.92733901 4.0204986  3.35102358 4.85204477 2.68043147]
 [1.70860419 1.74185962 1.05577583 2.0666576  0.64087243]
 [0.67241183 0.72956299 0.         1.69889996 0.7022981 ]
 ...
 [0.4720451  0.54728277 0.9849149  2.00898652 1.57025232]
 [0.47313337 0.48517626 1.11876217 1.83582826 1.7667507 ]
 [0.40127978 0.45391916 0.99232093 1.89168755 1.61632416]]


[[-0.22039745 -1.1351799 ]
 [ 0.3001803  -0.92449927]
 [-0.16222452  0.09711075]
 [ 0.91851359  0.17948312]
 [-0.70936471  0.50478613]]




In [ ]:
features = X
N, D = features.shape
k = 5
num_iters = 1
idxs = np.random.choice(N, size=k, replace=False)
centers = features[idxs]
assignments = np.zeros(N, dtype=int)

for n in range(num_iters):
    ### YOUR CODE HERE
    updated = False         # check update

    for i in range(10):
        # find square dist to each center
        square_dist = (features[n] - centers) ** 2
        square_dist = np.sum(square_dist, axis=-1)

        print(f"{square_dist}\n")

        # assign to the closest center
        cluster_idx = np.argmin(square_dist)
        if cluster_idx != assignments[i]:
            updated = True
            assignments[i] = cluster_idx

    # compute new center
    centers = np.zeros((k, D))
    cluster_cnt = np.zeros((k, 1))
    for i in range(N):
        centers[assignments[i]] += features[i]
        cluster_cnt[assignments[i]] += 1
    
    print(cluster_cnt)
    centers = centers / cluster_cnt

    # check assignment
    if not updated:
        break
    ### END YOUR CODE

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[ 7.73374682  5.72588098 16.16388691 20.62364452 12.04002622]

[[390.]
 [ 10.]
 [  0.]
 [  0.]
 [  0.]]


C:\Users\16567\AppData\Local\Temp\ipykernel_29616\1929274086.py:34: RuntimeWarning: invalid value encountered in divide
  centers = centers / cluster_cnt


In [ ]:


# idxs = np.random.choice(N, size=k, replace=False)
# centers = features[idxs]
# assignments = np.zeros(N, dtype=int)

# for n in range(num_iters):
#     ### YOUR CODE HERE
#     for i in range(N):
#         assignments[i] = np.argmin(((features[i] - centers) ** 2).mean(axis=-1))
#     a = np.eye(k)[assignments]
#     a /= a.sum(axis=0)
#     centers = a.T.dot(features)
#     ### END YOUR CODE

idx = np.random.choice(100, 5, replace=False)
print(idx)

features = X
# print(features)

centers = features[idx]
print(centers)

[92 97 30 48 42]
[[-0.80228445 -0.50661511]
 [-0.83452539 -0.05424771]
 [-1.21265067 -0.11370069]
 [-0.9966796   0.56474184]
 [-0.52937333  0.5995328 ]]


In [ ]:
N, D = features.shape

k = 5

num_iters = 100

n = 0


# find dist (L2) to each center
dist = (features[n] - centers) ** 2

print("\n")
print(dist)

dist = np.sum(dist, axis=-1)
# dist = dist.sum(axis=-1)

print("\n")
print(dist)

assignments = np.argmin(dist)


print("\n")
print(dist)



[[7.24635544 2.2698891 ]
 [7.07381581 1.11143824]
 [5.20542172 1.24032923]
 [6.23755817 0.18944967]
 [8.79013784 0.16037398]]


[9.51624454 8.18525405 6.44575095 6.42700784 8.95051182]


[9.51624454 8.18525405 6.44575095 6.42700784 8.95051182]


In [ ]:
N, D = features.shape

k = 5

num_iters = 100

# Randomly initalize cluster centers
idxs = np.random.choice(N, size=k, replace=False)
centers = features[idxs]
assignments = np.zeros(N, dtype=int)

print(type(features))

print(features[0])
print("\n")
print(centers)
print("\n")
print(features[0] - centers)
print("\n")
print(((features[0] - centers) ** 2))
print("\n")
print((0.65053404+1.47589589)/2)
print("\n")
print(((features[0] - centers) ** 2).mean(axis=-1))

for n in range(num_iters):
    ### YOUR CODE HERE
    for i in range(N):
        assignments[i] = np.argmin(((features[i] - centers) ** 2).mean(axis=-1))
    a = np.eye(k)[assignments]
    a /= a.sum(axis=0)
    centers = a.T.dot(features)
    ### END YOUR CODE

<class 'numpy.ndarray'>
[-3.49419  1.     ]


[[ 1.41714771 -0.27904533]
 [-0.23965981 -1.01583798]
 [-0.36561699  1.24703653]
 [-0.7559384   0.26041488]
 [ 1.38733808  0.0205093 ]]


[[-4.9113377   1.27904533]
 [-3.25453019  2.01583798]
 [-3.12857301 -0.24703653]
 [-2.7382516   0.73958512]
 [-4.88152808  0.9794907 ]]


[[24.12123803  1.63595696]
 [10.59196674  4.06360277]
 [ 9.78796907  0.06102705]
 [ 7.49802181  0.54698615]
 [23.82931638  0.95940203]]


1.063214965


[12.8785975   7.32778475  4.92449806  4.02250398 12.3943592 ]
